# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [1]:
import twitter

CONSUMER_KEY = "7gxqNshXnmZyW898ZX2ZfokJl"
CONSUMER_SECRET = "imUILT5m2IwNmE9NWhoO0XBsVwuEpxxDLU40XqcN220W68KYXM"

ACCESS_TOKEN_KEY = "704633785384439808-bPBV0FKqGvXcJV3zRyo12IXPOvR4X5y"
ACCESS_TOKEN_SECRET = "GvMbNQ9EZMQqOqBdR7M2LhaLEEeaJuDYXncAWikt7v27N"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя. По каждому пользователю достаточно собрать 200 твитов.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [2]:
import time

In [3]:
def get_user_tweets(user_id):
    for i in range(20):
        try:
            twits = api.GetUserTimeline(user_id, count = 200)
            break
        except:
            time.sleep(2)
            if i == 19:
                return []
    ret  = list()
    tw = list()
    for a in twits:
        #if a.retweeted_status == None:
            ret.append(
            {'lang': a.lang, 
             'favorited': a.favorited, 
             'truncated': a.truncated, 
             'text': a.text,
             'retweeted_status': a.retweeted_status == None,
             'created_at': a.created_at, 
             'retweeted': a.retweeted, 
             'source': a.source, 
             'user': {'id': a.user.id}, 
             'id': a.id})
    # your code here
    return ret

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [4]:
import re

In [5]:
def get_words(text):
    """returns list of words"""
    res = re.split("[?/,.!:\"\;\'\n@#…\-=&()+‘’_* ]*", text)
    res1 = list()
    for x in res:
        if x != '' and len(x) > 2:
            res1.append(x)
    return res1

In [6]:
print get_words(""" *d'dggg/sxf*  **f """)

['dggg', 'sxf']


In [7]:
import nltk
#nltk.download()

Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [8]:
wnl = nltk.WordNetLemmatizer()

In [10]:
stw = set(nltk.corpus.stopwords.words())
def get_tokens(words):
    """returns list of tokens"""
    # your code here
    global stw
    ret = list()
    for a in words:
        a = a.lower()
        if not (a in stw):
            ret.append(wnl.lemmatize(a))
    return ret

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [11]:
def get_tweet_tokens(tweet):
    # your code here
    return get_tokens(get_words(tweet))

In [12]:
print get_tokens(get_words("here 'll different words"))

['different', u'word']


Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [13]:
import json

In [14]:
def make_addr(uid, addr):
    return addr + "/" + str(uid)

In [15]:
def read_from_file(uid, addr = "files/twits"):
    name = make_addr(uid, addr)
    with open(name) as f:
        for line in f:
            return json.loads(line)

In [16]:
import os

In [17]:
def save_to_file(tweets, uid, addr = "files/twits"):
    name = make_addr(uid, addr)
    if not os.path.exists(os.path.dirname(name)):
        try:
            os.makedirs(os.path.dirname(name))
        except:
            pass
    outfile = open(name, 'w')
    json.dump(tweets, outfile)
    outfile.close()
    

In [18]:
def filter_twits(twits):
    res = list()
    for x in twits:
        if x['retweeted_status'] and not("http" in x['text']):
            res.append(x)
    return res

In [19]:
def collect_users_tokens(df_users, download = False):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    users = list()
    users_tokens = list()
    for x in df_users.uid:
        #print x
        tokens = dict()
        try:
            tweets = read_from_file(x)
            tweets = filter_twits(tweets)
            #save_to_file(tweets, x, addr = "files/filtred_twits")
            print "in file"
        except:
            if download:
                print "from net"
                tweets = get_user_tweets(x)
                tweets = filter_twits(tweets)
                save_to_file(tweets, x)
            else:
                tweets = []
        
        for y in tweets:
            twtok = get_tweet_tokens(y['text'])
            for tok in twtok:
                try:
                    tokens[tok] += 1
                except: 
                    tokens[tok] = 1
        users.append(x)
        users_tokens.append(tokens)
    return users, users_tokens


In [20]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer


TRAINING_SET_PATH = "train.csv"
TESTING_SET_PATH = "test.csv"
df_users_train = pd.read_csv(TRAINING_SET_PATH, sep=",", header=0)
df_users_ex = pd.read_csv(TESTING_SET_PATH, sep=",", header=0)
df_users_ex['cls'] = None
df_users = pd.concat([df_users_train, df_users_ex])
print "start"
users, users_tokens = collect_users_tokens(df_users, download = True)
v = DictVectorizer()
vs = v.fit_transform(users_tokens)

start
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in file
in

In [21]:
import numpy as np
np.savez("files/out_4.dat", data=vs, users=users, users_tokens=users_tokens )

In [34]:
vsT = vs.T
c = vsT.getcol(0)
for x in xrange(1,vsT.shape[1]):
    c += vsT.getcol(x)
c = c.toarray()

In [35]:
fn = v.get_feature_names()

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(vs)

In [89]:
j = 0
d = list()
for x in fn:
    d.append((x, int(c[j][0])))
    j += 1

In [90]:
from  wordcloud import WordCloud as WC

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [53]:
def draw_tag_cloud(v, vs, words = 100, h = 500, w = 1000, addr = "files/cloud.png"):
    """Draws tag cloud of found tokens"""
    vsT = vs.T
    c = vsT.getcol(0)
    for x in xrange(1,6898):
        c += vsT.getcol(x)
    c = c.toarray()
    
    fn = v.get_feature_names()
    j = 0
    d = list()
    for x in fn:
        d.append((x, int(c[j][0])))
        j += 1
    wordcloud = WC(width = w, height = h, max_words = words).generate_from_frequencies(d)
    wordcloud.background_color = 'white'
    image = wordcloud.to_image()
    image.save(addr)
    return

In [54]:
draw_tag_cloud(v, vs)